Start of Question 3

In [2]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scikit-learn scipy numpy matplotlib -verbose

Using pip 23.3.2 from f:\Anaconda\Lib\site-packages\pip (python 3.11)
Note: you may need to restart the kernel to use updated packages.


ERROR: rbose is not a valid editable requirement. It should either be a path to a local project or a VCS URL (beginning with bzr+http, bzr+https, bzr+ssh, bzr+sftp, bzr+ftp, bzr+lp, bzr+file, git+http, git+https, git+ssh, git+git, git+file, hg+file, hg+http, hg+https, hg+ssh, hg+static-http, svn+ssh, svn+http, svn+https, svn+svn, svn+file).


In [39]:
import sys
if sys.version_info[0] < 3:
	raise Exception("Python 3 not detected.")
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from scipy import io

mnistData = np.load(f"../data/mnist-data.npz")
print("\nloaded %s data!" % "minst")
fields = "test_data", "training_data", "training_labels"
for field in fields:
    print(field, mnistData[field].shape)
mnistTest = mnistData["test_data"]
mnistAllData = mnistData["training_data"]
mnistAllLabels = mnistData["training_labels"]

spamData = np.load(f"../data/spam-data.npz")
print("\nloaded %s data!" % "spam")
fields = "test_data", "training_data", "training_labels"
for field in fields:
    print(field, spamData[field].shape)
spamTest = spamData["test_data"]
spamAllData = spamData["training_data"]
spamAllLabels = spamData["training_labels"]

print(spamAllLabels)


loaded minst data!
test_data (10000, 1, 28, 28)
training_data (60000, 1, 28, 28)
training_labels (60000,)

loaded spam data!
test_data (1000, 32)
training_data (4171, 32)
training_labels (4171,)
[1 1 1 ... 0 0 0]


In [5]:
print(mnistAllLabels[5])

5


In [6]:
#Test out permutation function
#Seeding rng
rng = np.random.default_rng(seed=888)
a = np.array([1,2,3,4])
b = np.array([4,3,2,1])

indexes = np.arange(len(a))
rng.shuffle(indexes)
print(indexes)

a = a[indexes]
b = b[indexes]

print(a, b)

[1 0 3 2]
[2 1 4 3] [3 4 1 2]


In [40]:
rng = np.random.default_rng(seed=888)

#Shuffle Datasets
randIndex = rng.permutation(len(mnistAllLabels))

# #Pre-shuffled value
# print("\n Pre-shuffled value \n")
# print(mnistAllData[0])
# print(mnistAllLabels[0])

# print("\n Pre-shuffled value on index\n")
# print(mnistAllData[randIndex[0]])
# print(mnistAllLabels[randIndex[0]])

#ShuffleValues
mnistAllLabels = mnistAllLabels[randIndex]
mnistAllData = mnistAllData[randIndex]

# #Post-shuffled values should still align
# print("\n Post-shuffled value \n")
# print(mnistAllData[0])
# print(mnistAllLabels[0])

randIndex = rng.permutation(len(spamAllLabels))
spamAllLabels = spamAllLabels[randIndex]
spamAllData = spamAllData[randIndex]

In [43]:
#Partition Array
mnistValLabels = mnistAllLabels[0:10000]
mnistValData = mnistAllData[0:10000]

mnistTrainLabels = mnistAllLabels[10000:]
mnistTrainData = mnistAllData[10000:]

spamSplit = round(len(spamAllData) * 0.2)
print(spamSplit)
spamValLabels = spamAllLabels[0:spamSplit]
spamValData = spamAllData[0:spamSplit]

spamTrainLabels = spamAllLabels[spamSplit:]
spamTrainData = spamAllData[spamSplit:]

print(len(mnistTrainData), len(mnistValData))
print(len(mnistTrainLabels), len(mnistValLabels))

print(len(spamTrainData), len(spamValData))
print(len(spamTrainLabels), len(spamValLabels))

834
50000 10000
50000 10000
3337 834
3337 834


In [44]:
#Classification Accuracy
def classAccuracy(predictionList, truthList):
    score = 0
    for (pred, truth) in zip(predictionList, truthList):
        if pred == truth:
            score += 1
    return score/len(predictionList)


Start of Question 4

In [22]:
#Create Classifier
minstClf = svm.SVC()

#Flatten minst
# print("Original Data")
# print(mnistTrainData.shape)
# print(mnistTrainData[0])

minstTrainFlat = np.reshape(mnistTrainData, (50000, 784))
minstValFlat = np.reshape(mnistValData, (10000, 784))

# print("Flattened Data")
# print(minstFlat.shape)
# print(minstFlat[0])

#Fit points
trainingPoints = [100, 200, 500, 1000, 2000, 5000, 10000]
for numPoints in trainingPoints:
    minstClf.fit(minstTrainFlat[0:numPoints], mnistTrainLabels[0:numPoints])

    predictedTraining = minstClf.predict(minstTrainFlat[0:numPoints])
    predictedVal = minstClf.predict(minstValFlat)

    print(f"{numPoints}-dp training data score:", classAccuracy(predictedTraining, mnistTrainLabels[0:numPoints]))
    print(f"{numPoints}-dp validation data score:", classAccuracy(predictedVal, mnistValLabels))


100-dp training data score: 0.99
100-dp validation data score: 0.6495
200-dp training data score: 0.985
200-dp validation data score: 0.8104
500-dp training data score: 0.986
500-dp validation data score: 0.877
1000-dp training data score: 0.981
1000-dp validation data score: 0.919
2000-dp training data score: 0.985
2000-dp validation data score: 0.9338
5000-dp training data score: 0.9844
5000-dp validation data score: 0.949
10000-dp training data score: 0.9866
10000-dp validation data score: 0.96


In [49]:
#Create Classifier
spamClf = svm.SVC()

#Flatten spam
print("Original Data")
print(spamTrainData.shape)
print(spamTrainData[10])

# minstTrainFlat = np.reshape(mnistTrainData, (50000, 784))
# minstValFlat = np.reshape(mnistValData, (10000, 784))
# print("Flattened Data")
# print(minstFlat.shape)
# print(minstFlat[0])
#print(spamValLabels)

#Fit points
trainingPoints = [100, 200, 500, 1000, 2000, 3337]
for numPoints in trainingPoints:
    spamClf.fit(spamTrainData[0:numPoints], spamTrainLabels[0:numPoints])

    predictedTraining = spamClf.predict(spamTrainData[0:numPoints])
    predictedVal = spamClf.predict(spamValData)

    print(f"{numPoints}-dp training data score:", classAccuracy(predictedTraining, spamTrainLabels[0:numPoints]))
    print(f"{numPoints}-dp validation data score:", classAccuracy(predictedVal, spamValLabels))



Original Data
(3337, 32)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 2. 2. 0. 0. 0. 0.]
100-dp training data score: 0.78
100-dp validation data score: 0.7326139088729017
200-dp training data score: 0.8
200-dp validation data score: 0.7613908872901679
500-dp training data score: 0.81
500-dp validation data score: 0.7673860911270983
1000-dp training data score: 0.808
1000-dp validation data score: 0.7745803357314148
2000-dp training data score: 0.8145
2000-dp validation data score: 0.7793764988009593
3337-dp training data score: 0.8022175606832485
3337-dp validation data score: 0.7841726618705036
